In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

file_path = "F:\\Python\\PPG\\All data\\Patient_Signal\\PPG.csv"
data = pd.read_csv(file_path, header=None)

data.columns = ['Name', 'i', 'Bad', 'hands', 'label', 'systolic peak', 'Diastolic peak', 'Cardiac cycle', 'Cycle Area', 'SSI', 'Peak to Valley', 'Systolic peak y', 'Delta_T', '1st Derivative cycle', '1st Derivative peak', 'Ratio_BA', 'Ratio_CA', 'Ratio_DA', 'Ratio_BDCE_A', 'Ratio_CDB_A']
# Display the first few rows of the dataframe to understand its structure
data.head()

,Name,i,Bad,hands,label,systolic peak,Diastolic peak,Cardiac cycle,Cycle Area,SSI,Peak to Valley,Systolic peak y,Delta_T,1st Derivative cycle,1st Derivative peak,Ratio_BA,Ratio_CA,Ratio_DA,Ratio_BDCE_A,Ratio_CDB_A
0,伍黃秀霞,1,Right,Left,1,1278.0,0.390019,993.0,315.004855,987.0,720.0,0.516570,-1277.609981,1000.0,0.447513,1.022991,1.029483,1.047664,-2.109080,1.022221
1,伍黃秀霞,1,Right,Right,1,1265.0,0.275706,977.0,267.261331,995.0,714.0,0.485722,-1264.724294,994.0,0.327676,0.999176,0.997674,0.993834,-1.982910,1.000930
2,伍黃秀霞,3,Right,Left,1,1280.0,0.344179,997.0,307.701319,1011.0,718.0,0.468073,-1279.655821,994.0,0.427002,1.027341,1.050224,1.074771,-2.195227,1.027423
3,伍黃秀霞,3,Right,Right,1,1278.0,0.350065,979.0,286.925793,1011.0,705.0,0.506998,-1277.649935,1005.0,0.343972,0.991613,0.974413,0.961078,-1.887815,0.991555
4,伍黃秀霞,5,Right,Left,1,1296.0,0.354134,1000.0,318.004937,1009.0,701.0,0.428415,-1295.645866,996.0,0.360314,1.029600,1.045462,1.071122,-2.174579,1.028988


In [52]:
from sklearn.model_selection import KFold

grouped_data = data.groupby(['Name', 'hands', 'label']).size().reset_index(name='counts')
print(grouped_data)


    Name  hands  label  counts
0   伍黃秀霞   Left      1     136
1   伍黃秀霞  Right      1     256
2    劉正國   Left      1     186
3    劉正國  Right      1     182
4   吳劉碧桃   Left      1     101
..   ...    ...    ...     ...
83  陳高碧緞  Right      0     142
84  馮丁美珠   Left      1      88
85  馮丁美珠  Right      1      89
86  高陳賽金   Left      1     227
87  高陳賽金  Right      1     202

[88 rows x 4 columns]


In [56]:
#import kfold
# Ensure kf is defined
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# Apply KFold to the grouped data
for train_index, test_index in kf.split(grouped_data):
    train_data = grouped_data.iloc[train_index]
    test_data = grouped_data.iloc[test_index]
    common_train_data = pd.merge(train_data, data, on=['Name', 'hands', 'label'])
    common__test_data = pd.merge(test_data, data, on=['Name', 'hands', 'label'])
    x_train = common_train_data.iloc[:, 6:]
    x_test = common__test_data.iloc[:, 6:]
    y_train = common_train_data['label']
    y_test = common__test_data['label']
    
    # svm 避開分類都為同一類的問題
    if len(np.unique(y_train)) == 1:
        continue
    
    # Create the SVM model with class weights
    model = SVC(kernel='rbf', class_weight='balanced')
    # Fit the model to the training data
    model.fit(x_train, y_train)
    # Predict the labels for the test set
    y_pred = model.predict(x_test)

    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    # Calculate the accuracy
    accuracy = np.trace(cm) / np.sum(cm)
    print('Accuracy:', accuracy)
    print('Confusion Matrix:\n', cm)

Accuracy: 0.2268301478543094
Confusion Matrix:
 [[ 573    6]
 [2138   56]]
Accuracy: 0.38153998025666336
Confusion Matrix:
 [[ 703   57]
 [1196   70]]
Accuracy: 0.4717912552891396
Confusion Matrix:
 [[806 700]
 [798 532]]
Accuracy: 0.3795214245965498
Confusion Matrix:
 [[416 180]
 [935 266]]
Accuracy: 0.5472604934315924
Confusion Matrix:
 [[1607   34]
 [1379  101]]
